In [1]:
from scipy.cluster.vq import kmeans2
import numpy as np

In [20]:
X = np.random.normal(size=(10,4))**2
kmeans2(X,5,minit='points')[0]

array([[1.03169724, 0.50462342, 0.45129808, 0.6693754 ],
       [0.14532552, 2.26316298, 0.33034787, 0.06129624],
       [0.60518762, 7.55764718, 0.11321654, 0.92894233],
       [4.33849006, 0.15480039, 5.41459554, 0.02759707],
       [0.89924305, 1.43493841, 0.01404542, 6.10883047]])